In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
require 'Adversarials.exploitabilityVS'
require 'Nn.StrategyCriterion'
require 'csvigo'

In [2]:
function strategies_initializer(params, starting_ranges,num_iter,num_skip)
    local timer = torch.Timer()
    local num_iter = num_iter or 200
    local num_skip = num_skip or 50
    local params = params
    local tree_train= builder:build_tree(params)
    local tree_data = TreeData(tree_train)

    --- running CFR for nn_iter skiping num_skip
    local solver_train = TreeCFR2()
    solver_train.cfr_skip = num_skip
    solver_train:run_clean_cfr(tree_train,starting_ranges,num_iter)

    --- Getting training set:
    tree_data:get_training_set(tree_train,1)
    print('training train tree with CFR: ')--- ..timer:time().. ' secs')

    -- Building the neural net model
    local strategy_nn = StrategyNN()
    local nn_model = strategy_nn.model

    -- Building trainer:
    local nn_trainer = NNTrainer(tree_data,nn_model)
    -- Options:
    local opt = {}
    opt.learningRate = 0.001
    opt.momentum = 0.95
    opt.validate = false
    local current_criterion = StrategyLoss()---nn.SmoothL1Criterion()
    local train,test= nn_trainer:train(nn_model,current_criterion,opt,10000)
    print("training neural network: ")--- ..timer:time().. ' secs')

    --- NNWarm Starting
    local treeNNWarm = builder:build_tree(params)
    local dataNN = TreeData(treeNNWarm)
    dataNN:get_training_set(treeNNWarm,1)
    dataNN:generate_new_initial_strategies(treeNNWarm,nn_trainer)

    ---Warm start the regrets for NN:
    dataNN:warm_start_regrets(treeNNWarm,tree_train)
    print("target warm-start")

    --- Target Warm Starting
    local treeTargetWarm = builder:build_tree(params)
    local tree_data_target = TreeData(treeTargetWarm)
    tree_data_target:warm_start_targets_and_regrets(treeTargetWarm,tree_train)
    print("Whole Process: ")--- ..timer:time().. ' secs')
    ---timer:stop()
    return treeNNWarm,treeTargetWarm
end


function get_returns(tree1,tree2,num_iter)
    --- default number of iterations:
    local num_iter = num_iter or 100

    --- Evaluator : Poker match for num_iter times each combination
    local evaluator = StrategyEvaluator(tree1,tree2)
    evaluator:play_all_combinations_n_times(tree1,tree2,num_iter)
    local win_rate = evaluator.A2_winning_rate
    local avg_gain= evaluator.A2_avg_pot_won

    --- Return the avg pot won and the win rate:
    return avg_gain,win_rate
end


function get_tensor_returns(tree1,tree2,number_of_tests,num_iter)

    --- num_iter is the number of games played all 60 combinations:
    local num_iter = num_iter or 100
    local number_of_tests = number_of_tests or 10
    local avg_gain_tensor = torch.FloatTensor(1,number_of_tests)
    local win_rate_tensor = torch.FloatTensor(1,number_of_tests)

    --- Playing for number_of_tests times the 60 combinations num iter times
    for i =1,number_of_tests do
        local avg_gain,win_rate = get_returns(tree1,tree2,num_iter) 
        avg_gain_tensor[{1,i}]= avg_gain
        win_rate_tensor[{1,i}]= win_rate
    end
    print('avg gains: '..avg_gain_tensor:mean()..' ,win rate :'..win_rate_tensor:mean())
    return avg_gain_tensor,win_rate_tensor
end

function get_CFR_tensor_returns(tree1,tree2,num_tests,num_games,cfr_iter,cfr_skip)
    --- Cfr iterations and number of skpis:
    local cfr_iter = cfr_iter or 100
    local cfr_skip = cfr_skip or 50

    --- CFR Solver:
    local solver = TreeCFR2()
    solver.cfr_skip = cfr_skip 
    local starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
    starting_ranges[1]:copy(card_tools:get_uniform_range(tree2.board))
    starting_ranges[2]:copy(card_tools:get_uniform_range(tree2.board))

    --- Clean CFR : without exploitability computation
    ---local tree_data2 = TreeData(tree2)
---    local solver_train = TreeCFR2()
    solver:run_clean_cfr(tree2, starting_ranges,cfr_iter)

    ---print("CFR completed iterations")
    --- Starts poker_match
    local num_games = num_games or 100
    local num_tests = num_tests or 10
    local avg_gain_tensor = torch.FloatTensor(1,num_tests)
    local win_rate_tensor = torch.FloatTensor(1,num_tests)

    for i =1,num_tests do
        local avg_gain,win_rate = get_returns(tree1,tree2,num_games) 
        avg_gain_tensor[{1,i}]= avg_gain
        win_rate_tensor[{1,i}]= win_rate

    end
    print('avg gains: '..avg_gain_tensor:mean()..' ,win rate :'..win_rate_tensor:mean())
    return avg_gain_tensor,win_rate_tensor,solver
end


In [3]:
--- Important variables:
pot = 200
total_combinations = 60

In [39]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{pot,pot}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
--- exploitability
---exploit = TreeValues()

In [51]:
--- Initialize trees
nn_iter_init = 20
nn_iter_skip = 15
tree_bench = builder:build_tree(params)
tree_unif = builder:build_tree(params)
tree_NN, tree_targ,tree_train= strategies_initializer(params,starting_ranges,nn_iter_init,nn_iter_skip)

training train tree with CFR: 


training neural network: 


target warm-start


Whole Process: 


In [32]:
num_tests = 5
num_games = 100

In [33]:
local cfr_iter = 100
tree_unif1 = builder:build_tree(params)
solver1 = TreeCFR2()
solver1.cfr_skip = 50
solver1:run_cfr(tree_unif1, starting_ranges,cfr_iter)
---a1,b1 = get_tensor_returns(tree_bench,tree_unif1,num_tests,num_games)

In [34]:
local cfr_iter = 500
tree_unif2 = builder:build_tree(params)
solver2 = TreeCFR2()
solver2.cfr_skip = 400
solver2:run_cfr(tree_unif2, starting_ranges,cfr_iter)

In [35]:
a,b = get_tensor_returns(tree_unif1,tree_unif2,num_tests,num_games)

avg gains: -0.00026666651829146 ,win rate :0.50486981868744


In [36]:
print(solver1.exploitability_vec,solver2.exploitability_vec)

 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
  21.0075
   8.1652
   5.6186
   3.7363
   3.0580
   2.6466
   2.7383
[torch.FloatTensor of size 15]
	 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
 164.9537
   0.9191
   0.4792
   0.5076
   0.2924
   0.3513
   0.2917
   0.3158
[torch.FloatTensor of size 15]



In [37]:
---a2,b2 = get_tensor_returns(tree_bench,tree_unif2,num_tests,num_games)

In [47]:
num_tests =10
num_games = 100

In [48]:
--- Gains Before
gain_unif_before,win_unif_before=get_tensor_returns(tree_bench,tree_unif,num_tests,num_games)
gain_nn_before,win_nn_before=get_tensor_returns(tree_bench,tree_NN,num_tests,num_games)
gain_target_before,win_target_before=get_tensor_returns(tree_bench,tree_targ,num_tests,num_games)

avg gains: -0.0055499997339211 ,win rate :0.50091035366058


avg gains: 0.66606668233871 ,win rate :0.61289432644844


avg gains: 0.69851666092873 ,win rate :0.61887218356133


In [49]:
cfr_iter = 20
cfr_skip = 10

In [50]:
gain_unif_after,win_unif_after = get_CFR_tensor_returns(tree_bench,tree_unif,num_tests,num_games,cfr_iter,cfr_skip)
gain_nn_after,win_nn_after = get_CFR_tensor_returns(tree_bench,tree_NN,num_tests,num_games,cfr_iter,cfr_skip)
gain_target_after,win_target_after = get_CFR_tensor_returns(tree_bench,tree_targ,num_tests,num_games,cfr_iter,cfr_skip)

avg gains: 0.67951667308807 ,win rate :0.61953836083412


avg gains: 0.60031667351723 ,win rate :0.57670522928238


avg gains: 0.61846667528152 ,win rate :0.58098089694977


In [ ]:
gb = gain_unif_before:cat(gain_nn_before,1):cat(gain_target_before,1)
ga = gain_unif_after:cat(gain_nn_after,1):cat(gain_target_after,1)
wb = win_unif_before:cat(win_nn_before,1):cat(win_target_before,1)
wa = win_unif_after:cat(win_nn_after,1):cat(win_target_after,1)
res = gb:cat(ga,1):cat(wb,1):cat(wa,1)
res_table = torch.totable(res)
csvigo.save("res_match_last.csv",res_table)

In [ ]:
local num_iter = torch.range(1,gain_unif_before:size(2))
local plot = Plot()

plot:line(num_iter,gain_unif_before:view(-1),'black' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,gain_unif_after:view(-1),'black' ,'No Warm start CFR = Uniform warm start')
---plot:line(num_iter,gain_nn_before:view(-1),'blue' ,'No Warm start CFR = Uniform warm start')
---plot:line(num_iter,gain_target_before:view(-1),'red' ,'No Warm start CFR = Uniform warm start')
plot:legend(true):title('Average Gain per Match')
plot:xaxis('Number of iterations'):yaxis('Loss'):redraw()

In [ ]:
local num_iter = torch.range(1,win_unif_before:size(2))
local plot = Plot()

plot:line(num_iter,win_unif_before:view(-1),'black' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,win_nn_after:view(-1),'blue' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,win_target_after:view(-1),'red' ,'No Warm start CFR = Uniform warm start')
plot:xaxis('Number of iterations'):yaxis('Loss'):redraw()

In [ ]:
local num_iter = torch.range(1,gain_unif_before:size(2))
local plot = Plot()

plot:line(num_iter,gain_unif_before:view(-1),'black' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,gain_nn_after:view(-1),'blue' ,'No Warm start CFR = Uniform warm start')
plot:line(num_iter,gain_target_before:view(-1),'red' ,'No Warm start CFR = Uniform warm start')
plot:xaxis('Number of iterations'):yaxis('Loss'):redraw()

In [ ]:
--- Initialize trees
nn_iter_init = 5
nn_iter_skip = 0
tree_bench2 = builder:build_tree(params)
tree_unif2 = builder:build_tree(params)
tree_NN2, tree_targ2 = strategies_initializer(params,starting_ranges,nn_iter_init,nn_iter_skip)
--- Gains Before
gain_unif_before2,win_unif_before2 = get_tensor_returns(tree_bench2,tree_unif2,num_tests,num_games)
gain_nn_before2,win_nn_before2 = get_tensor_returns(tree_bench2,tree_NN2,num_tests,num_games)
gain_target_before2,win_target_before2 = get_tensor_returns(tree_bench2,tree_targ2,num_tests,num_games)

In [ ]:
solver_train = TreeCFR2()
solver_train.cfr_skip = 0
solver_train.number_of_explo_points = 10
t1,t2= solver_train:run_match_cfr(tree_unif, starting_ranges,tree_bench ,200,10,50)

In [ ]:
print(t1:mean())
print(t2:mean())

In [ ]:
gain_tensor,win_rate_tensor = get_final_results(tree_bench,tree_targ,10,50)

In [ ]:
print(gain_tensor:std())
print(win_rate_tensor:std())

In [ ]:
function get_all_results(tree1,tree2,num_cfr_iter,num_repetitions,num_games_per_card)
    local num_cfr_iter = num_cfr_iter or 100
    local num_repetitions =  num_repetitions or 10
    local num_games_per_card = num_games_per_card or 50
    local = interval = num_cfr_iter/15
    
    for i in 
    
    
    
end

In [ ]:
torch.round(400/15)

In [ ]:
x = torch.range(1,400,28)

In [ ]:
x:size(1)

In [ ]:
num_iter = 200
match_NN_targ= StrategyEvaluator(tree_targ,tree_NN)
match_NN_targ:play_all_combinations_n_times(tree_targ,tree_NN,num_iter)
match_NN_targ:display_results()
sd=match_NN_targ.A2_pot_tensor:clone():div(60*num_iter*pot):std()
print("pots won: " ,match_NN_targ.A2_total_pot)
print("avg number of pots won: " ,match_NN_targ.A2_avg_pot_won)
print("standard deviation, ",sd )
print("win_rate", match_NN_targ.A2_winning_rate)

In [ ]:
num_iter = 20
match_NN_targ= StrategyEvaluator(tree_targ,tree_NN)
match_NN_targ:play_all_combinations_n_times(tree_targ,tree_NN,num_iter)
match_NN_targ:display_results()
sd=match_NN_targ.A2_pot_tensor:clone():div(60*num_iter*pot):std()
print("pots won: " ,match_NN_targ.A2_total_pot)
print("avg number of pots won: " ,match_NN_targ.A2_avg_pot_won)
print("standard deviation, ",sd )
print("win_rate", match_NN_targ.A2_winning_rate)

In [ ]:
num_iter = 500
match_NN_targ= StrategyEvaluator(tree_targ,tree_NN)
match_NN_targ:play_all_combinations_n_times(tree_targ,tree_NN,num_iter)
match_NN_targ:display_results()
sd=match_NN_targ.A2_pot_tensor:clone():div(60*num_iter*pot):std()
print("pots won: " ,match_NN_targ.A2_total_pot)
print("avg number of pots won: " ,match_NN_targ.A2_avg_pot_won)
print("standard deviation, ",sd )
print("win_rate", match_NN_targ.A2_winning_rate)

In [ ]:
local num_iter = 100
match_NN_targ= StrategyEvaluator(tree_bench,tree_targ)
match_NN_targ:play_all_combinations_n_times(tree_bench,tree_targ,num_iter)
match_NN_targ:display_results()
print("avg number of pots won: " ,match_NN_targ.A2_total_pot/(total_combinations*num_iter*pot))

In [ ]:
local num_iter = 100
match_NN_targ= StrategyEvaluator(tree_NN,tree_targ)
match_NN_targ:play_all_combinations_n_times(tree_NN,tree_targ,num_iter)
match_NN_targ:display_results()
print("avg number of pots won: " ,match_NN_targ.A2_total_pot/(total_combinations*num_iter*pot))

In [ ]:
local num_iter = 500
match_NN_targ= StrategyEvaluator(tree_unif,tree_targ)
match_NN_targ:play_all_combinations_n_times(tree_unif,tree_targ,num_iter)
match_NN_targ:display_results()
print("avg number of pots won: " ,match_NN_targ.A2_total_pot/(total_combinations*num_iter*pot))

In [ ]:
cards = {}
cards.P1 = 1
cards.P2 = 3
cards.board = 4
number_of_games = 100

In [ ]:
match_NN_targ= StrategyEvaluator(tree_unif,tree_targ)
match_NN_targ:play_cards_n_times(tree_unif,tree_targ,number_of_games,cards)
match_NN_targ:display_results()
print("avg number of pots won: " ,match_NN_targ.A2_pot)

In [ ]:
torch.FloatTensor(1,6):fill(0)

In [ ]:
solver_train = TreeCFR2()
solver_train.cfr_skip = 50
solver_train:run_cfr(tree_unif,starting_ranges,200)

In [ ]:
treeNNWarm_cfr = TreeCFR2()
treeNNWarm_cfr.cfr_skip = 0
treeNNWarm_cfr:run_cfr(treeNNWarm, starting_ranges,number_of_iterations)
treeTargetWarm_cfr = TreeCFR2()
treeTargetWarm_cfr.cfr_skip = 0
treeTargetWarm_cfr:run_cfr(treeTargetWarm, starting_ranges,number_of_iterations)

In [ ]:
print("board: ", match_NN_targ.board)
print("card1: ", match_NN_targ.card1)
print("card2: ", match_NN_targ.card2)

print("pot_before",match_NN_targ.A2_pot)
match_NN_targ:play_game(tree_bench,tree_NN,1)
print("pot_after",match_NN_targ.A2_pot)

print("pot_before",match_NN_targ.A2_pot)
match_NN_targ:play_game(tree_NN,tree_bench,2)
print("pot_after",match_NN_targ.A2_pot)

In [ ]:
---visualiser = TreeVisualiser()
---visualiser:graphviz(tree1, "zero_blind")

In [ ]:
solver5 = TreeCFR2()
solver5.cfr_skip = 20
solver5:run_cfr(tree_500,starting_ranges,500)

In [ ]:
solver2 = TreeCFR2()
solver2.cfr_skip = 20
solver2:run_cfr(tree_200,starting_ranges,50)

In [ ]:
--- NNWarm Starting
dataNN = TreeData(tree_200)
dataNN:get_training_set(tree_200,1)

dataNN5 = TreeData(tree_500)
dataNN5:get_training_set(tree_500,1)



In [ ]:
tree_eval_200 = StrategyEvaluator(tree_bench,tree_200)
tree_eval_500 = StrategyEvaluator(tree_bench,tree_500)
tree_eval = StrategyEvaluator(tree_200,tree_500)


In [ ]:
tree_eval_200:play_all_combinations_n_times(tree_bench,tree_200,20)
tree_eval_200:display_results()

In [ ]:
tree_eval_500:play_all_combinations_n_times(tree_bench,tree_500,20)
tree_eval_500:display_results()

In [ ]:
tree_eval:play_all_combinations_n_times(tree_200,tree_500,50)
tree_eval:display_results()

In [ ]:
avg_pot_won = tree_eval:calculate_return(tree_200,tree_500,500,50)

In [ ]:
---timer = torch.Timer()
--print('Time elapsed ' .. timer:time().real .. ' seconds')

In [ ]:
avg_pot_won2 = tree_eval_200:calculate_return(tree_bench,tree_200,200,50)

In [ ]:
avg_pot_won5 = tree_eval_500:calculate_return(tree_bench,tree_500,200,50)

In [ ]:
local plot = Plot()
local x = tree_eval_200.range_games

plot:line(x,x:clone():fill(avg_pot_won2:mean()),'red' ,'Original CFR in the paper')
plot:line(x,avg_pot_won2:view(-1),'brown' ,'No Warm start CFR')

plot:line(x,x:clone():fill(avg_pot_won5:mean()),'blue' ,'Original CFR in the paper')
plot:line(x,avg_pot_won5:view(-1),'black' ,'No Warm start CFR')

plot:draw()



In [ ]:
tree_eval:display_results()

In [ ]:
print(tree_eval.A2_pot_matrix:sum())
print(tree_eval.A2_pot_matrix:std())

In [ ]:
print("Total number of pots won : ",tree_eval.A2_pot_matrix:sum()/min_pot)

In [ ]:
print("average amount won per game",tree_eval.A2_pot_matrix:sum()/(number_of_times*60))

In [ ]:
avg_per_hand = tree_eval.A2_pot_matrix:mean(2)

In [ ]:
plot = Plot():circle(torch.range(1,avg_per_hand:size(1)),avg_per_hand:view(-1))
plot:legend(true):title('Player 2 pot distribution, 10000 games, each possible card dealt'):draw()


In [ ]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [ ]:
cards = {}
cards.P1 = 1
cards.P2 = 2
cards.board = 4
number_of_games = 100

In [ ]:
tree_eval.A2_pot_tensor

In [ ]:
print(tree_eval.A2_pot_tensor:mean())
print(tree_eval.A2_pot_tensor:std())

In [ ]:
tree_eval:display_results()

In [ ]:
local pot_table = {}
local size_sample = 50
for i = 1,50 do
    tree_eval:play_several_games(tree1,tree2,size_sample)
    table.insert(pot_table,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table):mean())

In [ ]:
pot_table_A2 = {}
local number_of_games = 1000
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    local cards = distribute_cards(cards_tensor[i])
    tree_eval:play_deterministic_games(tree1,tree2,number_of_games,cards)
    table.insert(pot_table_A2,tree_eval.A2_pot)
end
print("player_2 won: ",torch.FloatTensor(pot_table_A2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table_A2):std())
tree_eval:display_results()

In [ ]:
torch.sqrt(60)*(1803/150716)

In [ ]:
plot = Plot():histogram(torch.FloatTensor(pot_table_A2)):draw()

In [ ]:
pot_table1 = {}
local number_of_games = 10
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 20
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 50
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)   --- --> CHECK NUMBER OF GAMES!
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
 cardsInv = {}
cardsInv.P1 = cards.P2
cardsInv.P2 = cards.P1
cardsInv.board = cards.board

In [ ]:
pot_table2 = {}
local size_sample = 100
for i = 1,20 do
    tree_eval:play_deterministic_games(tree1,tree2,size_sample,cardsInv)
    table.insert(pot_table2,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table2):std())

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table1)):draw()

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table2)):draw()

In [ ]:
x[1]

In [ ]:
      local winner = self:compute_winner(tree_current_agent)
      if agent_to_act == 1 then
        self.current_winner = winner
        print("winner", winner)
        self:update_pot(winner,tree_current_agent)
      else
        self.current_winner = winner%2+1
        print("winner", winner)
        self:update_pot(winner,tree_current_agent)
      end


In [ ]:
number_of_times = 100
interval = 20
range_games = torch.range(0,number_of_times,interval)
pots_won_tensor = torch.FlaoatTensor(1,range_games:size(1))
avg_gain_per_game = torch.FloatTensor(1,range_games:size(1))
range_games[1] = 1
for i = 1,range_games:size(1) do
    print("iteration: ", range_games[i])
    tree_eval:play_all_combinations_n_times(tree_bench,tree_200,range_games[i])
    local pot_won = tree_eval.A2_pot_matrix:sum()/(pot*range_games[i]*60)
    local avg_gain = tree_eval.A2_pot_matrix:sum()/(range_games[i]*60)
    pots_won_tensor[{1,i}] = pot_won
    avg_gain_per_game[{1,i}] = avg_gain
    print("Total number of pots won : ",pot_won)
    print("average amount won per game",avg_gain)
    collectgarbage()
end
